In [12]:
from scipy.optimize import linprog
from math import sqrt

import random
import copy

random.seed(42)

In [2]:
def solve_price_matrix(pred_treat_effect, ab_matrix):
    # verify dimensions -- need to have a, b for each treatment
    assert(len(pred_treat_effect[0]) == len(ab_matrix[0]) == len(ab_matrix[1]))
    
    price_matrix = copy.deepcopy(pred_treat_effect)
    
    # the price p(i,t) is a linear function of the input 
    for i in range(len(pred_treat_effect)):
        for j in range(len(pred_treat_effect[0])):
            price_matrix[i][j] = ab_matrix[0][j]*pred_treat_effect[i][j] + ab_matrix[1][j]
            
    return price_matrix

In [3]:
def solve_demand_matrix(wtp_matrix, price_matrix, budget_list):
    # verify dimensions
    # same number of subjects
    assert(len(wtp_matrix) == len(price_matrix) == len(budget_list))
    # same number of treatments
    assert(len(wtp_matrix[0]) == len(price_matrix[0]))
    
    prob_matrix = []
    
    # for feasability condition
    feasability = [[1 for x in row] for row in wtp_matrix]
    feasability_budget = [1 for x in budget_list]
    x_bounds = (0, 1)
    
#     feasability = [[1, 1, 1], [1, 1, 1], [1, 1, 1]]
#     feasability_budget = [1, 1, 1]
    
    # convert wtp_matrix to negative for maximization using linprog
    wtp_temp = [[-x for x in row] for row in wtp_matrix]
    
    # solve LP problem row by row
    for i, row in enumerate(wtp_temp):
        res = linprog(wtp_temp[i], A_ub=price_matrix[i], b_ub=budget_list[i],
                  A_eq=feasability, b_eq=feasability_budget, 
                  bounds=(x_bounds,)*len(wtp_temp[0]))
        if (res.success == False):
            print "linprog not successful"
        prob_matrix.append(res.x.tolist())
    
    return prob_matrix

In [4]:
def solve_excess_demand(demand_matrix, capacity_list):
    # make sure the dimensions make sense
    # we have a capacity for each treatment
    assert (len(demand_matrix[0]) == len(capacity_list))
    
    # sum along the columns of demand_matrix to get total demand
    total_demand_list = [0]*len(capacity_list)
    for i in demand_matrix:
        for j, val in enumerate(i):
            total_demand_list[j] += val
    
    # subtract to get excess demand
    excess_demand = [x1 - x2 for x1, x2 in zip(total_demand_list, capacity_list)]
    return excess_demand

In [9]:
def solve_clearing_error(excess_demand_matrix):
    sum_squares = sum([x**2 for x in excess_demand_matrix])
    return sqrt(sum_squares)

In [14]:
def get_ab_matrix(budget_list):
    a_list = [random.randint(-b, 0) for b in budget_list]
    b_list = [random.randint(-b, b) for b in budget_list]
    return [a_list, b_list]

In [75]:
def get_nbr_ab_values(ab_matrix, excess_demand_list, k):
    nbr = [[(ab_matrix[0][t] + excess_demand_list[t]*k) for t in range(len(excess_demand_list))], 
           [(ab_matrix[1][t] + excess_demand_list[t]*k) for t in range(len(excess_demand_list))]]
    print nbr
    
def get_nbr_clear_price_tuple(pred_treat_effect, ab_matrix, excess_demand_list, k_list):
    nbr_price = solve_price_matrix(pred_treat_effect, nbr_ab_val)
    

In [79]:
ab_matrix = [[1, 1], [2, 2]]
excess_demand_list = [5, 5]
k_list = [2**i for i in range(10)]

print [excess_demand_list[t]*2 for t in range(len(excess_demand_list))]
print len(ab_matrix[0])
print len(excess_demand_list)

get_nbr_ab_values(ab_matrix, excess_demand_list, 1)

[10, 10]
2
2
[[6, 6], [7, 7]]


In [37]:
# matrix of WTP -- subjects along rows, treatments along columns
wtp_matrix = [[3, 1, 1], [1, 4, 1], [1, 1, 5]]

# matrix of predicted treatment effect -- subjects along rows, treatments along columns
pred_treat_effect = [[1, 1, 1], [1, 2, 2], [1, 3, 3]]

# list of factors to get different nbr prices
k_list = [2**i for i in range(10)]

budget_list = [100, 100, 100]
capacity_list = [1, 1, 1]

In [38]:
ab_matrix = get_ab_matrix(budget_list)
price_matrix = solve_price_matrix(pred_treat_effect, ab_matrix)
demand_matrix = solve_demand_matrix(wtp_matrix, price_matrix, budget_list)
excess_demand_list = solve_excess_demand(demand_matrix, capacity_list)
clearing_error = solve_clearing_error(excess_demand_list)

print ab_matrix
print price_matrix
print demand_matrix
print excess_demand_list
print clearing_error

[[-8, -13, -70], [28, 22, -70]]
[[20, 9, -140], [20, -4, -210], [20, -17, -280]]
[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]
[0.0, 0.0, 0.0]
0.0
